- Ler duas plantas (1ª e 2ª ordem) via multiplexador (CD4052BE)
- Escolher quais canais de cada planta usar (via `A[n]B[m]`)
- Escolher qual planta mostrar na Serial (`SHOW1`, `SHOW2`, ou `SHOWBOTH`)

### CÓDIGO EXPLICADO

>```cpp
>#include <Arduino.h>
>#include <freertos/FreeRTOS.h>
>#include <freertos/task.h>
>#include <freertos/timers.h>
>```

Inclui as bibliotecas do Arduino e do FreeRTOS para multitarefas e timers.

>```cpp
>static const TickType_t samplingInterval = 20 / portTICK_PERIOD_MS;
>static const TickType_t timeToStartInterval = 5000 / portTICK_PERIOD_MS;
>```

Define dois intervalos de tempo:
- `samplingInterval`: intervalo de leitura do sensor (20 ms)
- `timeToStartInterval`: atraso para aplicar o "step" (5 segundos)

>```cpp
>static const uint8_t A_PIN = 14;
>static const uint8_t B_PIN = 27;
>```

Define os pinos de controle `A` e `B` do CD4052 (multiplexador)

>```cpp
>static const uint8_t SENSOR_PIN_1 = 15; // Planta 1ª ordem
>static const uint8_t SENSOR_PIN_2 = 34; // Planta 2ª ordem
>```
Define os pinos ADC (entrada analógica) usados para ler os sinais das plantas

>```cpp
>static const uint8_t MV_PIN = 25;
>```

Pino de saída para aplicar o sinal de controle (step) via DAC

>```cpp
>volatile float VCC = 0.0;
>volatile uint16_t sensorReadingInt1;
>volatile uint16_t sensorReadingInt2;
>volatile float sensorVoltage1;
>volatile float sensorVoltage2;
>```

Variáveis globais:
- `VCC`: valor da tensão usada no DAC (3.3 V quando ativa)
- `sensorReadingIntX`: valor lido (ADC bruto)
- `sensorVoltageX`: valor convertido para tensão

>```cpp
>int seletor_1ordem = 1;
>int seletor_2ordem = 1;
>int modoExibicao = 1;  // 1 = mostrar planta 1ª ordem, 2 = mostrar 2ª ordem
>```
Define os canais ativos de cada planta e qual delas será mostrada na Serial.

>```cpp
>static TimerHandle_t getSensorReadingTimer = NULL;
>static TimerHandle_t stepInputStartTimer = NULL;
>static TimerHandle_t muxConfigTimer = NULL;
>```
Cria os timers FreeRTOS para:
- Leitura periódica (`getSensorReadingTimer`)
- Aplicar o step (`stepInputStartTimer`)
- Configurar o MUX (`muxConfigTimer`)

### 📦 Função `setMUX()`
>```cpp
>void setMUX() {
>  int a = 0, b = 0;
>```
Inicia os bits de controle A e B

>```cpp
>switch (seletor_1ordem) {
>    case 1: a = 0; b = 0; break;
>    case 2: a = 1; b = 0; break;
>    case 3: a = 0; b = 1; break;
>    case 4: a = 1; b = 1; break;
>    default: a = 0; b = 0; break;
>  }
>```
Define o par de bits A/B para o canal desejado da planta de 1ª ordem.
O CD4052 usa esses mesmos bits para comutar os dois canais simultaneamente (1ª e 2ª ordem).

>```cpp
>digitalWrite(A_PIN, a);
>  digitalWrite(B_PIN, b);
>```
Escreve os valores nos pinos de controle.

>```cpp
>Serial.print("MUX ajustado para Planta 1ª ordem canal: ");
>  Serial.print(seletor_1ordem);
>  Serial.print(" | Planta 2ª ordem canal: ");
>  Serial.println(seletor_2ordem);
>}
>```
Mostra qual canal de cada planta está ativo.

### ⏱ Função `getSensorReadingCallback`
>```cpp
>void getSensorReadingCallback(TimerHandle_t xTimer) {
>```
Callback disparado periodicamente pelo timer de leitura.

>```cpp
> sensorReadingInt1 = analogRead(SENSOR_PIN_1);
>  sensorVoltage1 = sensorReadingInt1 * (VCC / 4096.0);
>```
Lê o valor da planta 1ª ordem e converte para tensão.
